In [1]:
!pip install wordfreq
from wordfreq import word_frequency
import spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 54.3 MB/s eta 0:00:00
  Attempting uninstall: regex
    Found existing installation: regex 2023.6.3
    Uninstalling regex-2023.6.3:
      Successfully uninstalled regex-2023.6.3


In [2]:
def check_freq(sentence):
  nlp = spacy.load("en_core_web_sm")
  words = [tok.lemma_ for tok in nlp(sentence) if tok.pos_ not in ["PUNCT", "SPACE"]]
  freq_dict = {}
  for word in words:
    freq = word_frequency(word, 'en')
    freq_dict[word] = freq
  vocab = dict(sorted(freq_dict.items(), key=lambda item: item[1]))
  return vocab

In [3]:
### Words Frequency
import pandas as pd
def compare_sentence_freq(sentence1, sentence2):
    freq_dict1 = check_freq(sentence1)
    freq_dict2 = check_freq(sentence2)
    low_freq_count1 = sum(1 for freq in freq_dict1.values() if freq < 1e-4)
    low_freq_count2 = sum(1 for freq in freq_dict2.values() if freq < 1e-4)
    return 0 if low_freq_count1 > low_freq_count2 else 1

def compare_min_word_freq(sentence1, sentence2):
    freq_dict1 = check_freq(sentence1)
    freq_dict2 = check_freq(sentence2)
    min_freq1 = min(freq_dict1.values()) if freq_dict1 else float('inf')
    min_freq2 = min(freq_dict2.values()) if freq_dict2 else float('inf')
    return 0 if min_freq1 < min_freq2 else 1

df_expl = pd.read_csv('explanation.csv')
df_baseline = pd.read_csv('baseline.csv')
count_compare_sentence_freq = 0
count_compare_min_word_freq = 0
for (sentence1, sentence2) in zip(df_expl.iloc[:, 1], df_baseline.iloc[:, 3]):
    count_compare_sentence_freq += compare_sentence_freq(sentence1, sentence2)
    count_compare_min_word_freq += compare_min_word_freq(sentence1, sentence2)

rate_compare_sentence_freq = count_compare_sentence_freq / len(df_expl)
rate_compare_min_word_freq = count_compare_min_word_freq / len(df_expl)
print("Sucess rate for compare_sentence_freq:", rate_compare_sentence_freq)
print("Sucess rate for compare_min_word_freq:", rate_compare_min_word_freq)

Sucess rate for compare_sentence_freq: 0.9292929292929293
Sucess rate for compare_min_word_freq: 0.8383838383838383


In [4]:
def average_sentence_length(text):
    sentences = text.split('.')
    word_counts = [len(sentence.split()) for sentence in sentences if sentence.strip()]
    if len(word_counts) == 0:
        return 0
    return sum(word_counts) / len(word_counts)

In [5]:
### Sentence Length
def compare_average_sentence_length(sentence1, sentence2):
    avg_len1 = average_sentence_length(sentence1)
    avg_len2 = average_sentence_length(sentence2)
    return 0 if avg_len1 > avg_len2 else 1

df_expl = pd.read_csv('explanation.csv')
df_baseline = pd.read_csv('baseline.csv')
results = [compare_average_sentence_length(sentence1, sentence2)
           for sentence1, sentence2 in zip(df_expl.iloc[:, 1], df_baseline.iloc[:, 3])]
rate_of_1 = sum(results) / len(results)
print("Sucess rate:", rate_of_1)

Sucess rate: 0.5858585858585859


In [12]:
### Readability
!pip install textstat
import textstat
import pandas as pd

def compare_readability(sentence1, sentence2):
    score1 = textstat.flesch_kincaid_grade(sentence1)
    score2 = textstat.flesch_kincaid_grade(sentence2)
    return 0 if score1 > score2 else 1

df_expl = pd.read_csv('explanation.csv')
df_baseline = pd.read_csv('baseline.csv')
results = [compare_readability(sentence1, sentence2)
           for sentence1, sentence2 in zip(df_expl.iloc[:, 1], df_baseline.iloc[:, 3])]
rate_of_1 = sum(results) / len(results)
print("Sucess rate:", rate_of_1)

Sucess rate: 0.8888888888888888


In [9]:
!pip install lexicalrichness

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lexicalrichness: filename=lexicalrichness-0.5.1-py3-none-any.whl size=15414 sha256=f68299915784cafb399df96e570d37ea7c7b16628477babc02a147724be5cb04
  Stored in directory: /root/.cache/pip/wheels/cd/ba/80/d4dabc1bf242a672ffc00226a2303a7471bb841c0872b2c212
Successfully built lexicalrichness


In [16]:
### Lexical Diversity
from lexicalrichness import LexicalRichness
import pandas as pd

def calculate_lexical_diversity_measures(sentence):
    lex = LexicalRichness(sentence)
    hdd_draws = min(lex.words, 50)
    return {
        'ttr': lex.ttr,  # Type-Token Ratio
    }

def compare_lexical_diversity(sentence1, sentence2, measure):
    diversity1 = calculate_lexical_diversity_measures(sentence1)
    diversity2 = calculate_lexical_diversity_measures(sentence2)
    return 0 if diversity1[measure] > diversity2[measure] else 1

df_expl = pd.read_csv('explanation.csv')
df_baseline = pd.read_csv('baseline.csv')
measures = ['ttr']
for measure in measures:
    results = [compare_lexical_diversity(sentence1, sentence2, measure)
               for sentence1, sentence2 in zip(df_expl.iloc[:, 1], df_baseline.iloc[:, 3])]
    rate_of_1 = sum(results) / len(results)
    print(f"Sucess rate for {measure.upper()}: {rate_of_1}")

Sucess rate for TTR: 0.5555555555555556


In [ ]:
###Manually Measured Criterion:
##1. Accuracy and Factual Correctness: Both are accurate and align with the fact.
##2. Relevance and Completeness: All our explanations are better for English beginner. It also includes some facts and explain hard words which make sentences easier to understand. All include key points.
##3. Scores: our main models success rate is 97%.